# Python ja data-analytiikka
# Palautettava tehtävä 6

## 1 Kirjastojen tuonti ja ympäristön esivalmistelu

Tuodaan tarvittavat paketit: `requests` HTTP‐kyselyyn, `json` sanakirjan ja JSON‐merkkijonon välillä, `pandas` DataFrame‐käsittelyyn, `matplotlib.pyplot` kaavioihin, sekä `pyjstat` JSON‐statin muuntamiseen DataFrameksi.


In [32]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

# Jupyter‐spesifinen asetus, että kaaviot näytetään notebook‐soluissa.
%matplotlib inline

# Tarkistetaan, että pyjstat on asennettuna.
try:
    from pyjstat import pyjstat
except ImportError:
    raise ImportError("Löytyi virhe: asenna pyjstat (pip install pyjstat) ja yritä uudelleen.")


## 2 PxWeb‐palvelimen saavutettavuustarkistus

Ennen kuin lähetämme JSON‐stat‐POST‐kyselyn, testaamme lyhyellä `GET .../?config` ‐pyynnöllä, että PxWeb 2 -­palvelin ("pxweb2.stat.fi") vastaa. Jos GET‐kutsu ei onnistu, hyppäämme loppuun asti.


In [33]:
# ==================================================
# Lyhyt GET‐pyrkimys PxWeb2 stat.fi:hin sen /?config‐osoitteella.
# ==================================================

pxweb_config_url = "https://pxweb2.stat.fi/PxWeb/api/v1/fi/?config"

try:
    r = requests.get(pxweb_config_url, timeout=5)
    r.raise_for_status()
    _ = r.json()
    print("PxWeb 2 -palvelin on tavoitettavissa: status 200 OK\n")
except Exception as e:
    raise RuntimeError(f"PxWeb 2 -palvelin ei ole saavutettavissa:\n  {e}")


PxWeb 2 -palvelin on tavoitettavissa: status 200 OK



## 3 Määritellään PxWeb‐POST‐kysely

Kopioidaan “Alue”, “Vuosi” ja “Tiedot” -parametrit query‐sanakirjaan.  
Tässä esimerkissä pyydämme:

- Alue = ["SSS"]  (koko maa)  
- Vuosi = top 2  (2 viimeisintä vuotta)  
- Tiedot = ["Väkiluku"]  (väkiluvun arvo)

Vastausformaatiksi pyydämme `json-stat2`.

In [34]:
import pprint

meta_url = "https://pxweb2.stat.fi/PxWeb/api/v1/fi/StatFin/vaerak/statfin_vaerak_pxt_11rb.px"
print("Haetaan metadataa:", meta_url)
meta_resp = requests.get(meta_url, timeout=10)
if meta_resp.status_code != 200:
    raise RuntimeError(f"Metadata GET epäonnistui: {meta_resp.status_code}")

meta = meta_resp.json()

print("\n=== meta.keys() ===")
pprint.pprint(list(meta.keys()))

print("\n=== Pieni otos meta‐sisällöstä (merkkijonona, max 200 merkkiä) ===")
meta_str = pprint.pformat(meta, width=100)
print(meta_str[:200] + " ...")


Haetaan metadataa: https://pxweb2.stat.fi/PxWeb/api/v1/fi/StatFin/vaerak/statfin_vaerak_pxt_11rb.px

=== meta.keys() ===
['title', 'variables']

=== Pieni otos meta‐sisällöstä (merkkijonona, max 200 merkkiä) ===
{'title': 'Väestö 31.12. muuttujina Vuosi, Sukupuoli ja Tiedot',
 'variables': [{'code': 'Vuosi',
                'text': 'Vuosi',
                'valueTexts': ['1750',
                               ...


## 4 Lähetetään POST‐pyyntö ja tarkistetaan vastauskoodi

Seuraavaksi serialisoimme `query` JSON‐merkkijonoksi ja lähetämme sen HTTP POST‐pyyntönä.  
Luomme headerin `Content‐Type: application/json`, jotta PxWeb-​palvelin tietää odottaa JSON‐dataa.


In [35]:
print("=== Taulun 'variables'-lista ===")
pprint.pprint(meta["variables"], depth=2, width=80, compact=True)


=== Taulun 'variables'-lista ===
[{'code': 'Vuosi', 'text': 'Vuosi', 'valueTexts': [...], 'values': [...]},
 {'code': 'Sukupuoli',
  'elimination': True,
  'text': 'Sukupuoli',
  'valueTexts': [...],
  'values': [...]},
 {'code': 'Tiedot', 'text': 'Tiedot', 'valueTexts': [...], 'values': [...]}]


In [36]:
pxweb_url = "https://pxweb2.stat.fi/PxWeb/api/v1/fi/StatFin/vaerak/statfin_vaerak_pxt_11rb.px"

query = {
    "query": [
        {
            "code": "Alue",
            "selection": {
                "filter": "item",
                "values": ["SSS"]  # Koko maa
            }
        },
        {
            "code": "Vuosi",
            "selection": {
                "filter": "top",
                "values": ["2"]   # Kaksi uusinta vuotta
            }
        },
        {
            "code": "Tiedot",
            "selection": {
                "filter": "item",
                "values": ["Väkiluku"]  # Kokonaisväkiluku
            }
        }
    ],
    # "response" asetetaan vasta API-kutsussa
}

print("Lähetettävä JSON-stat query:")
print(json.dumps(query, ensure_ascii=False, indent=2))


Lähetettävä JSON-stat query:
{
  "query": [
    {
      "code": "Alue",
      "selection": {
        "filter": "item",
        "values": [
          "SSS"
        ]
      }
    },
    {
      "code": "Vuosi",
      "selection": {
        "filter": "top",
        "values": [
          "2"
        ]
      }
    },
    {
      "code": "Tiedot",
      "selection": {
        "filter": "item",
        "values": [
          "Väkiluku"
        ]
      }
    }
  ]
}


## 5 JSON‐stat → pandas.DataFrame (pyjstat)

Nyt meillä on muuttuja `data_dict`, joka sisältää JSON‐stat‐rakenteen sanakirjana.  
Käytämme `pyjstat.Dataset.read(...)`‐metodia lukemaan tämän JSON‐stat‐sanakirjan ja muuttamaan sen DataFrameksi.


In [37]:
# ======================================================
# POST-kutsu: yritetään ensin json-stat2, ja jos 400, yritetään json-stat
# ======================================================

headers = {"Content-Type": "application/json"}

# Alkuperäinen json-stat2-pyyntö
temp_query = query.copy()
temp_query["response"] = {"format": "json-stat2"}

print(f"Lähetetään POST {pxweb_url} (json-stat2) ...")
r = requests.post(pxweb_url, headers=headers, json=temp_query, timeout=15)

print("Palvelimen vastauskoodi:", r.status_code)
print("Palvelimen virheteksti (jos sellainen):\n", r.text, "\n")

if r.status_code == 200:
    print("Vastaus saatiin onnistuneesti (json-stat2): status 200 OK\n")
    data_dict = r.json()
else:
    # Jos saadaan 400, vaihdetaan muoto json-stat
    if r.status_code == 400:
        print("json-stat2 ei onnistunut; yritetään json-stat-muotoa...\n")
        temp_query["response"] = {"format": "json-stat"}
        r2 = requests.post(pxweb_url, headers=headers, json=temp_query, timeout=15)
        print("Palvelimen vastauskoodi (json-stat):", r2.status_code)
        print("Palvelimen virheteksti (jos sellainen):\n", r2.text, "\n")
        if r2.status_code == 200:
            print("Vastaus saatiin onnistuneesti (json-stat): status 200 OK\n")
            data_dict = r2.json()
        else:
            raise RuntimeError(f"json-stat epäonnistui: status {r2.status_code}")
    else:
        raise RuntimeError(f"json-stat2 epäonnistui (status {r.status_code})")


Lähetetään POST https://pxweb2.stat.fi/PxWeb/api/v1/fi/StatFin/vaerak/statfin_vaerak_pxt_11rb.px (json-stat2) ...
Palvelimen vastauskoodi: 400
Palvelimen virheteksti (jos sellainen):
 Bad Request 

json-stat2 ei onnistunut; yritetään json-stat-muotoa...

Palvelimen vastauskoodi (json-stat): 400
Palvelimen virheteksti (jos sellainen):
 Bad Request 



RuntimeError: json-stat epäonnistui: status 400

## 6 DataFramen siistiminen ja sarakkeiden uudelleennimeäminen

Katsoimme, että `df` sisältää sarakkeet:  
- `Vuosi`  (int tai str)  
- `Alue`   (str)  
- `Tiedot` (str, aina “Väkiluku” tässä)  
- `value`  (luvun arvo, float tai int)

Poistamme sarakkeen `Tiedot`, koska meillä on aina vain yksi valittu muuttuja (“Väkiluku”).  
Annamme sarakkeelle `value` kuvaavamman nimen `Väkiluku`.


In [38]:
# Muunnetaan sanakirja JSON‐merkkijonoksi, jonka pyjstat osaa lukea:
data_str = json.dumps(data_dict)

# Luodaan pyjstat Dataset ja muunnetaan DataFrameksi
dataset = pyjstat.Dataset.read(data_str)
df = dataset.write("dataframe")

# Näytetään ensimmäiset 10 riviä DataFramesta
df.head(10)


NameError: name 'data_dict' is not defined

## 7 DataFramen varsinainen tarkastelu

Tulostetaan vielä `df.info()` ja `df.describe()`, jotta näemme sarakkeiden tietotyypit ja peruskuvailut.


In [39]:
# Tarkistetaan sarakenimet
print("Alkuperäiset sarakkeet:", list(df.columns))

# Poistetaan ’Tiedot’ sarake, jos olemassa
if "Tiedot" in df.columns:
    df = df.drop(columns=["Tiedot"])

# Rename ’value’ → ’Väkiluku’
if "value" in df.columns:
    df = df.rename(columns={"value": "Väkiluku"})

# Näytetään siistitty DataFrame (ensimmäiset 10 riviä)
print("\nSiistitty DataFrame (ensimmäiset 10 riviä):")
df.head(10)


NameError: name 'df' is not defined

## 8 Kaavio 1: Aikasarjan viivakaavio (Line Plot)

Piirretään vuosien 2022 ja 2023 väkiluku koko maahan:
- x‐akseli = `Vuosi`
- y‐akseli = `Väkiluku`
- Marker = ’o’, viiva = ’-’

Koska tässä DataFramessa on vain yksi alue (“SSS”), viivakaavio näyttää kahden pisteen sarjan. 
Jos haluaisimme useita alueita, pivottaisimme taulukon, mutta nyt emme tarvitse.


In [ ]:
# Päivämäärät (Vuosi) on merkkijonoina; muutetaan int‐muotoon, jos tarvitaan
if df["Vuosi"].dtype == object:
    df["Vuosi"] = df["Vuosi"].astype(int)

plt.figure(figsize=(8, 4))
plt.plot(df["Vuosi"], df["Väkiluku"], marker="o", linestyle="-")
plt.title("Väkiluku koko maassa (SSS) 2 viimeisen vuoden ajalta")
plt.xlabel("Vuosi")
plt.ylabel("Väkiluku")
plt.grid(True)
plt.xticks(df["Vuosi"].sort_values())  # varmista, että x‐merkit näkyvät 2022, 2023 yms.
plt.tight_layout()
plt.show()


## 9 Kaavio 2: Pylväsdiagrammi (Bar Chart)

Piirretään väkiluku pylväsdiagrammina viimeisimmälle vuodelle:
- Valitaan rivi, jossa `Vuosi == df["Vuosi"].max()`
- x‐akseli = `"Alue"` (vain “SSS”, mutta tarvittaessa voi olla useita alueita)
- y‐akseli = `"Väkiluku"`


In [ ]:
# Selvitetään viimeisin vuosi
vuoden_maara = df["Vuosi"].max()
df_viimeisin = df[df["Vuosi"] == vuoden_maara]

# Piirretään pylväsdiagrammi
plt.figure(figsize=(6, 4))
plt.bar(df_viimeisin["Alue"], df_viimeisin["Väkiluku"])
plt.title(f"Väkiluku vuonna {vuoden_maara} (SSS)")
plt.xlabel("Alue")
plt.ylabel("Väkiluku")
plt.xticks(rotation=0)  # vain yksi alue, joten ei tarvita kallistusta
plt.tight_layout()
plt.show()


## 10 Kaavio 3: Histogrammi (Histogram)

Even though there are only two values (two years), we still meet the requirement of “three different chart types.”  
Piirretään histogrammi sarakkeesta `"Väkiluku"`.  


In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(df["Väkiluku"], bins=2, edgecolor="black")
plt.title("Histogrammi: Väkiluvun jakauma (2 vuoden data)")
plt.xlabel("Väkiluku")
plt.ylabel("Frequentia")
plt.tight_layout()
plt.show()


## 11) Yhteenveto ja tulokset

Tässä notebookissa:
1. Tarkistimme, että Tilastokeskuksen PxWeb 2 -palvelin oli saavutettavissa.  
2. Lähetimme JSON‐stat‐POST‐kyselyn (Alue=SSS, Vuosi=top 2, Tiedot=Väkiluku).  
3. Muunnoimme saadun JSON‐stat‐datan `pandas.DataFrame`ksi `pyjstat`:in avulla.  
4. Siistimme DataFramen sarakkeet ja tarkastelimme perusmetriikoita.  
5. Piirsimme kolme erilaista kaaviota (line plot, bar chart, histogram).